In [ ]:
import os
import requests
import logging
import sqlite3
import pandas as pd
import re
import ssl
from datetime import datetime
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3 import PoolManager

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)
logging.getLogger("requests").setLevel(logging.WARNING)  # Disable logging from the requests module
#Change to the preferred directory
basedir = os.path.abspath(os.path.dirname(__file__))

### URLS
LOGIN_URL = 'https://www.tm3.com/homepage/login.jsf'
TAXEXEMPT_URL = 'https://www.tm3.com/mmdrewrite/mmd/14902.faces'
TAXABLE_URL = 'https://www.tm3.com/mmdrewrite/mmd/TaxableYieldCurveChanges.faces'

### Credentials
USERNAME = 'REDACTED'
PASSWORD = 'REDACTED'


# Define the scrape_data function here
def scrape_data(tax_feature, url):
    logger.info('Scraping %s at %s...' % (tax_feature, url))
    r = s.get(url)
    t = r.text
    soup = BeautifulSoup(t)
    if tax_feature == 'taxexempt':
        tgt_table = soup.find('table', {'class': 'data data14902'})
    else:  # 'taxable'
        tgt_table = soup.find('table', {'class': 'dataDL'})
    rows = tgt_table.findAll('tr')
    if len(soup.findAll('th')) > 0:
        rows = rows[2:]
    # get date
    dt = get_table_dt(soup, tax_feature)

    dates = []
    scales = []
    aaa_ylds = []
    aa_ylds = []
    a_ylds = []
    baa_ylds = []
    if tax_feature == 'taxexempt':
        for row in rows:
            cells = row.findAll('td')
            dates.append(dt)
            scales.append(cells[0].get_text())
            aaa_ylds.append(cells[2].get_text())
            aa_ylds.append(cells[5].get_text())
            a_ylds.append(cells[6].get_text())
            baa_ylds.append(cells[7].get_text())
    else:  # 'taxable'
        for row in rows:
            cells = row.findAll('td')
            dates.append(dt)
            scales.append(cells[1].get_text())
            aaa_ylds.append(cells[3].get_text())
            aa_ylds.append(cells[5].get_text())
            a_ylds.append(cells[7].get_text())
            baa_ylds.append(cells[9].get_text())
    d = {'date': dates, 'scale': scales, 'AAA': aaa_ylds, 'AA': aa_ylds, 'A': a_ylds, 'BAA': baa_ylds}
    df = pd.DataFrame(d)
    # reshape df for database insert
    df.mlt = pd.melt(df, id_vars=['date', 'scale'], var_name='rating')
    return df.mlt, tax_feature

# Define the get_table_dt function here
def get_table_dt(io, tax_feature):
    dt_str = io.find('h2', {'class': 'pgSubHeader'}).get_text()
    match = re.search(r'(\d+/\d+/\d+)', dt_str)
    dt_str = match.group(1)
    try:
        return datetime.strptime(dt_str, '%m/%d/%Y').strftime('%Y-%m-%d')
    except ValueError as e:
        logger.warning(e)

def insert_dframe(tablename, dframe, dbpath):
    db = sqlite3.connect(dbpath)
    curs = db.cursor()
    # Create the table if needed
    tblstr = 'CREATE TABLE IF NOT EXISTS %s (date TEXT, scale INTEGER, rating TEXT, value REAL, PRIMARY KEY (date, scale, rating))' % tablename
    if curs.execute(tblstr):
        logger.info('Table %s created or already exists...' % tablename)
    else:
        logger.error('Error creating table %s...' % tablename)
    # Insert data row by row, avoiding duplicates
    for _, row in dframe.iterrows():
        try:
            curs.execute('INSERT OR IGNORE INTO %s VALUES (?, ?, ?, ?)' % tablename, tuple(row))
        except sqlite3.IntegrityError as e:
            logger.info(e)
    db.commit()
    logger.info('Inserted %s rows into %s...' % (len(dframe), tablename))
    curs.close()
    db.close()


### Main Program
if __name__ == '__main__':
    ### Stuff required for a persistent session
    class MyAdapter(HTTPAdapter):
        def init_poolmanager(self, connections, maxsize, block):
            self.poolmanager = PoolManager(num_pools=connections, maxsize=maxsize, block=block, ssl_version=ssl.PROTOCOL_TLSv1)

    with requests.Session() as s:
        logging.info('Opening session...')
        s.mount('https://', MyAdapter())
        # Build payload that will eventually be posted to login
        payload = {}
        # Find hidden inputs
        """ uncomment line below if you run into any certificate verification issues """
        result = s.get(LOGIN_URL, verify=False)
        #result = s.get(LOGIN_URL)
        c = result.content
        soup = BeautifulSoup(c)
        hidden_inputs = soup.find_all('input', type='hidden')
        for hidden_input in hidden_inputs:
            name = hidden_input.get('name')
            payload[name] = hidden_input.get('value')
        payload['username'] = USERNAME
        payload['password'] = PASSWORD
        payload['loginButton'] = 'Login'
        logger.debug('Payload is %s' % payload)
        # Attempt to login
        logger.info('Attempting to login...')
        s.post(LOGIN_URL, data=payload, timeout=30.0)
        dbpath = os.path.join(basedir, os.pardir, 'ksm.db') #Change db to desired databse location
        df, tbl = scrape_data('taxexempt', TAXEXEMPT_URL)
        insert_dframe(tbl, df, dbpath)
        df, tbl = scrape_data('taxable', TAXABLE_URL)
        insert_dframe(tbl, df, dbpath)
        logger.info('Done...')

